<a href="https://colab.research.google.com/github/saivigneshmn/support-ticket/blob/main/rough/Analysis/Clusters_with_Product_Tags_and_Occurrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import spacy
from collections import Counter
import re

class TicketAnalyzer:
    def __init__(self):
        # Load SpaCy model for NLP tasks
        self.nlp = spacy.load('en_core_web_sm')

    def preprocess_text(self, text):
        """Preprocess text by cleaning and normalizing"""
        # Convert to lowercase and remove special characters
        text = re.sub(r'[^a-zA-Z\s]', ' ', str(text).lower())
        # Remove extra whitespace
        text = ' '.join(text.split())
        return text

    def combine_text_fields(self, subject, body):
        """Combine subject and body with appropriate weighting"""
        # Give more weight to subject by repeating it
        return f"{subject} {subject} {subject} {body}"

    def cluster_tickets(self, df, n_clusters=5):
        """Cluster tickets using TF-IDF and Agglomerative Clustering"""
        # Combine and preprocess text
        combined_texts = [
            self.combine_text_fields(
                self.preprocess_text(subject),
                self.preprocess_text(body)
            )
            for subject, body in zip(df['subject'], df['body'])
        ]

        # Create TF-IDF vectors
        vectorizer = TfidfVectorizer(
            max_features=1000,
            stop_words='english',
            ngram_range=(1, 2)
        )
        tfidf_matrix = vectorizer.fit_transform(combined_texts)

        # Reduce dimensionality with PCA
        pca = PCA(n_components=50)
        reduced_features = pca.fit_transform(tfidf_matrix.toarray())

        # Perform clustering
        clustering = AgglomerativeClustering(
            n_clusters=n_clusters,
            metric='euclidean'
        )
        clusters = clustering.fit_predict(reduced_features)

        return clusters, vectorizer

    def extract_product_tags(self, text):
        """Extract product-related entities and noun phrases"""
        doc = self.nlp(str(text))

        # Extract named entities
        entities = [ent.text for ent in doc.ents if ent.label_ in ['ORG', 'PRODUCT']]

        # Extract noun phrases that might be products
        noun_phrases = [chunk.text for chunk in doc.noun_chunks
                       if any(token.pos_ in ['PROPN', 'NOUN'] for token in chunk)]

        # Combine and clean
        all_products = entities + noun_phrases
        return list(set(all_products))

    def extract_issues(self, text):
        """Extract issues using dependency parsing and pattern matching"""
        doc = self.nlp(str(text))
        issues = []

        # Pattern 1: Look for problem-indicating words
        problem_indicators = ['issue', 'problem', 'error', 'failure', 'bug', 'crash']

        for token in doc:
            # Check for problem indicators
            if token.lower_ in problem_indicators:
                # Get the full phrase around the problem
                phrase = ' '.join([t.text for t in token.subtree])
                issues.append(phrase)

            # Check for negative verbs with subjects
            if token.dep_ == 'ROOT' and token.pos_ == 'VERB':
                if any(child.dep_ == 'neg' for child in token.children):
                    subject = next((child for child in token.children
                                  if child.dep_ == 'nsubj'), None)
                    if subject:
                        issues.append(f"{subject.text} {token.text}")

        return list(set(issues))

    def analyze_cluster_content(self, df, clusters, vectorizer):
        """Analyze the content of each cluster"""
        cluster_analysis = {}

        for cluster_id in set(clusters):
            cluster_mask = clusters == cluster_id
            cluster_docs = df[cluster_mask]

            # Combine all text in cluster
            all_text = ' '.join([
                self.combine_text_fields(str(subject), str(body))
                for subject, body in zip(cluster_docs['subject'], cluster_docs['body'])
            ])

            # Get most common terms
            feature_names = vectorizer.get_feature_names_out()
            cluster_tfidf = vectorizer.transform([all_text])
            top_indices = cluster_tfidf.toarray()[0].argsort()[-10:][::-1]
            top_terms = [feature_names[i] for i in top_indices]

            # Extract products and issues
            products = []
            issues = []
            for _, row in cluster_docs.iterrows():
                products.extend(self.extract_product_tags(
                    f"{row['subject']} {row['body']}"
                ))
                issues.extend(self.extract_issues(
                    f"{row['subject']} {row['body']}"
                ))

            # Calculate frequencies
            product_freq = Counter(products)
            issue_freq = Counter(issues)

            cluster_analysis[cluster_id] = {
                'size': sum(cluster_mask),
                'top_terms': top_terms,
                'products': dict(product_freq.most_common(5)),
                'issues': dict(issue_freq.most_common(5))
            }

        return cluster_analysis

def analyze_tickets(df):
    """Main function to analyze tickets"""
    analyzer = TicketAnalyzer()

    # Perform clustering
    clusters, vectorizer = analyzer.cluster_tickets(df)

    # Analyze clusters
    cluster_analysis = analyzer.analyze_cluster_content(df, clusters, vectorizer)

    # Add cluster assignments to dataframe
    df['cluster'] = clusters

    # Extract products and issues for each ticket
    df['extracted_products'] = df.apply(
        lambda x: analyzer.extract_product_tags(f"{x['subject']} {x['body']}"),
        axis=1
    )
    df['extracted_issues'] = df.apply(
        lambda x: analyzer.extract_issues(f"{x['subject']} {x['body']}"),
        axis=1
    )

    return df, cluster_analysis

In [ ]:
# Read the CSV file
df = pd.read_csv('your_tickets.csv')

# Process the data
from ticket_data_processor import load_ticket_data
from ticket_analysis import analyze_tickets

# Prepare and analyze the data
df_clean = load_ticket_data(df)
df_analyzed, cluster_analysis = analyze_tickets(df_clean)

# Display results
from ticket_data_processor import display_analysis_results
display_analysis_results(df_analyzed, cluster_analysis)

<ipython-input-6-474430a4015e>:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'cluster'] = clusters
<ipython-input-6-474430a4015e>:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'extracted_products'] = df.apply(


              id                                            subject  \
0  1001352387736  Urgent: Critical impact on enterprise network ...   
1  1004699418379     Intermittent Cursor Freezing Issue on Dell XPS   
5  1024619926727    Exchange Request for Malfunctioning Dell XPS 13   
6  1026487543175       Request for Server Administration Assistance   
7  1027850274374   Immediate Attention Required: AWS Outage Concern   

                                                body  \
0  Dear Customer Support Team, We are experiencin...   
1  Dear Customer Support,<br><br>I hope this mess...   
5  Dear Tech Online Store Customer Support,\n\nI ...   
6  Dear IT Services Customer Support, \n\nWe are ...   
7  Greetings IT Services Customer Support,\n\nI a...   

                                              answer      type  \
0  Subject: Re: Urgent: Critical impact on enterp...  Incident   
1  Dear <name>,\n\nThank you for reaching out reg...  Incident   
5  Dear <name>,\n\nThank you for reach

<ipython-input-6-474430a4015e>:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['extracted_issues'] = df.apply(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import spacy
from collections import Counter
import re
from spacy.matcher import DependencyMatcher

class TicketAnalyzer:
    def __init__(self, df, n_clusters=5):
        self.df = df[df['language'] == 'en'].copy()
        self.n_clusters = n_clusters
        self.nlp = spacy.load('en_core_web_sm')
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            stop_words='english',
            ngram_range=(1, 2)
        )

    def preprocess_text(self, text):
        text = str(text).lower()
        text = re.sub(r'[^\w\s]', ' ', text)
        return text

    def extract_issues(self, text):
        doc = self.nlp(str(text))
        issues = []

        # 1. Extract issues based on dependency patterns
        # Looking for noun phrases that are objects of negative verbs
        for sent in doc.sents:
            for token in sent:
                if token.dep_ == 'neg':
                    head = token.head
                    if head.pos_ == 'VERB':
                        # Get the object of the negative verb
                        for child in head.children:
                            if child.dep_ in ['dobj', 'pobj', 'attr']:
                                issue_span = doc[child.left_edge.i:child.right_edge.i + 1]
                                issues.append(issue_span.text)

        # 2. Extract technical terms followed by state descriptions
        for ent in doc.ents:
            if ent.label_ in ['PRODUCT', 'ORG', 'GPE']:
                # Look for adjectives or verbs following the entity
                entity_token = ent[-1]
                next_token = doc[min(entity_token.i + 1, len(doc) - 1)]
                if next_token.pos_ in ['ADJ', 'VERB']:
                    issues.append(f"{ent.text} {next_token.text}")

        # 3. Find comparative structures indicating problems
        for token in doc:
            if token.pos_ == 'ADJ' and token.dep_ == 'acomp':
                # Look for comparatives indicating degradation
                if any(child.dep_ == 'than' for child in token.children):
                    for child in token.head.children:
                        if child.dep_ in ['nsubj', 'nsubjpass']:
                            issues.append(f"{child.text} {token.text}")

        # 4. Extract subject-verb-object combinations where verb indicates state
        for token in doc:
            if token.pos_ == 'VERB' and token.dep_ == 'ROOT':
                subj = None
                obj = None
                for child in token.children:
                    if child.dep_ == 'nsubj':
                        subj = child
                    if child.dep_ in ['dobj', 'pobj']:
                        obj = child
                if subj and obj:
                    issues.append(f"{subj.text} {token.text} {obj.text}")

        # Remove duplicates and normalize
        unique_issues = list(set(issues))

        # Filter out too short or too long issues
        filtered_issues = [issue for issue in unique_issues
                         if 2 <= len(issue.split()) <= 5]

        return filtered_issues

    def analyze(self):
        # Combine subject and body for clustering
        combined_text = self.df['subject'] + ' ' + self.df['body']
        combined_text = combined_text.apply(self.preprocess_text)

        # Create TF-IDF matrix
        tfidf_matrix = self.vectorizer.fit_transform(combined_text)

        # Perform clustering
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=42)
        self.df['cluster'] = kmeans.fit_predict(tfidf_matrix)

        # Extract issues
        self.df['extracted_issues'] = self.df['body'].apply(self.extract_issues)

        # Analyze clusters
        cluster_info = []
        for cluster_id in range(self.n_clusters):
            cluster_tickets = self.df[self.df['cluster'] == cluster_id]

            # Get most common tags in cluster
            all_tags = []
            for _, row in cluster_tickets.iterrows():
                tags = [col for col in row.index if col.startswith('tag_') and pd.notna(row[col])]
                all_tags.extend([row[tag] for tag in tags])

            tag_counts = Counter(all_tags)
            total_tags = len(all_tags)
            tag_percentages = {tag: (count/total_tags)*100
                             for tag, count in tag_counts.most_common()}

            # Get common issues
            all_issues = []
            for issues in cluster_tickets['extracted_issues']:
                all_issues.extend(issues)
            common_issues = Counter(all_issues).most_common(3)

            # Get representative samples
            sample_tickets = cluster_tickets.sample(min(3, len(cluster_tickets)))

            cluster_info.append({
                'cluster_id': cluster_id,
                'size': len(cluster_tickets),
                'tag_percentages': tag_percentages,
                'common_issues': common_issues,
                'samples': sample_tickets[['subject', 'body']].values.tolist()
            })

        return cluster_info

def analyze_tickets(df):
    analyzer = TicketAnalyzer(df, n_clusters=5)
    return analyzer.analyze()

TOP TAGS %Occurenncies (not product particularly)

In [ ]:
# Assuming your dataframe is called 'df'
analyze_tickets(df)

[{'cluster_id': 0,
  'size': 43,
  'tag_percentages': {'Technical Support': 13.88888888888889,
   'Product Support': 13.88888888888889,
   'Problem Resolution': 10.714285714285714,
   'Hardware Failure': 9.126984126984127,
   'Software Bug': 5.952380952380952,
   'Urgent Issue': 5.555555555555555,
   'Warranty Claim': 5.158730158730158,
   'Technical Guidance': 5.158730158730158,
   'Service Recovery': 4.365079365079365,
   'Customer Service': 4.365079365079365,
   'General Inquiry': 3.1746031746031744,
   'Performance Tuning': 2.7777777777777777,
   'Sales Inquiry': 2.380952380952381,
   'Order Issue': 2.380952380952381,
   'Network Issue': 1.5873015873015872,
   'Returns and Exchanges': 1.1904761904761905,
   'Account Assistance': 1.1904761904761905,
   'System Maintenance': 0.7936507936507936,
   'Service Disruption': 0.7936507936507936,
   'Refund Request': 0.7936507936507936,
   'Service Notification': 0.7936507936507936,
   'Login Issue': 0.7936507936507936,
   'Feature Request':

# TEST with sample input extracting product tags and issues

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from collections import Counter
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Step 1: Preprocess Text
def preprocess_text(text):
    """Tokenize and clean the text for processing."""
    tokens = [token.lemma_.lower() for token in nlp(text) if token.is_alpha]
    return " ".join(tokens)

# Step 2: Extract Product Tags
def extract_product_tags(text):
    """Use NER to extract product-related entities."""
    doc = nlp(text)
    tags = [ent.text for ent in doc.ents if ent.label_ in {"PRODUCT", "ORG"}]
    return tags

# Step 3: Extract Issues
def extract_issues(text):
    """Extract potential issues using dependency parsing and POS tagging."""
    doc = nlp(text)
    issues = [
        token.text for token in doc
        if token.pos_ in {"NOUN", "ADJ"} and token.dep_ in {"amod", "dobj", "nsubj"}
    ]
    return issues

# Step 4: Cluster Tickets
def cluster_tickets(df, text_column, n_clusters=5):
    """Cluster tickets based on their content."""
    vectorizer = TfidfVectorizer(max_features=5000)
    tfidf_matrix = vectorizer.fit_transform(df[text_column])
    pca = PCA(n_components=3)
    reduced_data = pca.fit_transform(tfidf_matrix.toarray())

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(reduced_data)

    df["cluster_id"] = clusters
    return df, reduced_data

# Step 5: Calculate Tag Percentages
def calculate_tag_percentages(cluster_data, tag_column):
    """Calculate percentage of each tag in a cluster."""
    all_tags = [tag for tags in cluster_data[tag_column] for tag in tags]
    tag_counts = Counter(all_tags)
    total_tickets = len(cluster_data)
    return {tag: (count / total_tickets) * 100 for tag, count in tag_counts.items()}

# Full Processing Pipeline
def process_tickets(df):
    """Full pipeline to process tickets."""
    # Preprocess the text data
    df["subject_clean"] = df["subject"].apply(preprocess_text)
    df["body_clean"] = df["body"].apply(preprocess_text)

    # Extract product tags and issues
    df["product_tags"] = df["body"].apply(extract_product_tags)
    df["issues"] = df["body"].apply(extract_issues)

    # Combine subject and body for clustering
    df["combined_text"] = df["subject_clean"] + " " + df["body_clean"]

    # Cluster tickets
    df, reduced_data = cluster_tickets(df, "combined_text", n_clusters=5)

    # Calculate tag percentages for each cluster
    cluster_tag_stats = {}
    for cluster_id, cluster_data in df.groupby("cluster_id"):
        cluster_tag_stats[cluster_id] = calculate_tag_percentages(cluster_data, "product_tags")

    return df, reduced_data, cluster_tag_stats

# Example Usage
if __name__ == "__main__":
    # Example DataFrame
    data = {
        "subject": [
            "Urgent: AWS Deployment Issue",
            "Network issues with Cisco Router ISR4331",
            "Request for Jira Software setup",
            "Battery problem in Dell XPS",
            "MacBook Air screen flickering"
        ],
        "body": [
            "We are facing issues with AWS deployment. Immediate assistance required.",
            "The Cisco Router ISR4331 is frequently disconnecting.",
            "Please assist with setting up a new Jira Software project.",
            "The battery in my Dell XPS 13 is draining quickly.",
            "My MacBook Air screen flickers intermittently."
        ]
    }
    df = pd.DataFrame(data)

    # Process tickets
    processed_df, reduced_data, cluster_stats = process_tickets(df)

    # Output results
    print("Processed DataFrame:")
    print(processed_df[["subject", "product_tags", "issues", "cluster_id"]])
    print("\nCluster Tag Statistics:")
    for cluster_id, stats in cluster_stats.items():
        print(f"Cluster {cluster_id}: {stats}")


Processed DataFrame:
                                    subject                product_tags  \
0              Urgent: AWS Deployment Issue                          []   
1  Network issues with Cisco Router ISR4331  [The Cisco Router ISR4331]   
2           Request for Jira Software setup                          []   
3               Battery problem in Dell XPS                          []   
4             MacBook Air screen flickering               [MacBook Air]   

                            issues  cluster_id  
0  [issues, Immediate, assistance]           4  
1                        [ISR4331]           0  
2                   [new, project]           2  
3                        [battery]           3  
4                         [screen]           1  

Cluster Tag Statistics:
Cluster 0: {'The Cisco Router ISR4331': 100.0}
Cluster 1: {'MacBook Air': 100.0}
Cluster 2: {}
Cluster 3: {}
Cluster 4: {}


In [ ]:
import pandas as pd

# Replace 'helpdesk_customer_multi_lang_tickets.csv' with your file name
df = pd.read_csv('helpdesk_customer_multi_lang_tickets.csv')

# Preview the first few rows
print(df.head())


              id                                            subject  \
0  1001352387736  Urgent: Critical impact on enterprise network ...   
1  1004699418379     Intermittent Cursor Freezing Issue on Dell XPS   
2  1006966905046  Dringend: Unterstützung für die Datenwiederher...   
3  1009231330404           Anfrage zu den MacBook Air M1 Funktionen   
4  1024440081041  Solicitação de Assistência com Erro de Instala...   

                                                body  \
0  Dear Customer Support Team, We are experiencin...   
1  Dear Customer Support,<br><br>I hope this mess...   
2  Hallo, wir haben severe Datenverluste in MySQL...   
3  Sehr geehrtes Kundenserviceteam,\n\nich hoffe,...   
4  Caro Suporte ao Cliente,\n\nEstou enfrentando ...   

                                              answer      type  \
0  Subject: Re: Urgent: Critical impact on enterp...  Incident   
1  Dear <name>,\n\nThank you for reaching out reg...  Incident   
2  Hallo, vielen Dank, dass Sie uns ko

# only "en" lang
product retrivel and Issues Statistics/ Cluster Classification

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from collections import Counter
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Step 1: Preprocess Text
def preprocess_text(text):
    """Tokenize and clean the text for processing."""
    if pd.isna(text):
        return ""
    tokens = [token.lemma_.lower() for token in nlp(text) if token.is_alpha]
    return " ".join(tokens)

# Step 2: Extract Product Tags
def extract_product_tags(text):
    """Use NER to extract product-related entities."""
    if pd.isna(text):
        return []
    doc = nlp(text)
    tags = [ent.text for ent in doc.ents if ent.label_ in {"PRODUCT", "ORG"}]
    return tags

# Step 3: Extract Issues
def extract_issues(text):
    """Extract potential issues using dependency parsing and POS tagging."""
    if pd.isna(text):
        return []
    doc = nlp(text)
    issues = [
        token.text for token in doc
        if token.pos_ in {"NOUN", "ADJ"} and token.dep_ in {"amod", "dobj", "nsubj"}
    ]
    return issues

# Step 4: Cluster Tickets
def cluster_tickets(df, text_column, n_clusters=5):
    """Cluster tickets based on their content."""
    vectorizer = TfidfVectorizer(max_features=5000)
    tfidf_matrix = vectorizer.fit_transform(df[text_column])
    pca = PCA(n_components=3)
    reduced_data = pca.fit_transform(tfidf_matrix.toarray())

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(reduced_data)

    df["cluster_id"] = clusters
    return df, reduced_data

# Step 5: Calculate Tag Percentages
def calculate_tag_percentages(cluster_data, tag_columns):
    """Calculate percentage of each tag in a cluster."""
    all_tags = []
    for column in tag_columns:
        all_tags.extend(cluster_data[column].dropna().tolist())

    # Flatten the nested tags and count occurrences
    all_tags = [tag for sublist in all_tags for tag in sublist.split(",") if tag]
    tag_counts = Counter(all_tags)
    total_tickets = len(cluster_data)
    return {tag: (count / total_tickets) * 100 for tag, count in tag_counts.items()}

# Full Processing Pipeline
def process_tickets(df):
    """Full pipeline to process tickets."""
    # Filter only English tickets
    df = df[df["language"] == "en"].reset_index(drop=True)

    # Preprocess the text data
    df["subject_clean"] = df["subject"].apply(preprocess_text)
    df["body_clean"] = df["body"].apply(preprocess_text)

    # Extract product tags and issues
    df["product_tags"] = df["body"].apply(extract_product_tags)
    df["issues"] = df["body"].apply(extract_issues)

    # Combine subject and body for clustering
    df["combined_text"] = df["subject_clean"] + " " + df["body_clean"]

    # Cluster tickets
    df, reduced_data = cluster_tickets(df, "combined_text", n_clusters=5)

    # Calculate tag percentages for each cluster
    cluster_tag_stats = {}
    tag_columns = [f"tag_{i}" for i in range(1, 10)]
    for cluster_id, cluster_data in df.groupby("cluster_id"):
        cluster_tag_stats[cluster_id] = calculate_tag_percentages(cluster_data, tag_columns)

    return df, reduced_data, cluster_tag_stats

# Example Usage
if __name__ == "__main__":
    # Path to dataset
    file_path = "/content/helpdesk_customer_multi_lang_tickets.csv"

    # Load dataset
    df = pd.read_csv(file_path)

    # Process tickets (considering only English language)
    processed_df, reduced_data, cluster_stats = process_tickets(df)

    # Output results
    print("Processed DataFrame (English Only):")
    print(processed_df[["subject", "product_tags", "issues", "cluster_id"]])
    print("\nCluster Tag Statistics:")
    for cluster_id, stats in cluster_stats.items():
        print(f"Cluster {cluster_id}: {stats}")


Processed DataFrame (English Only):
                                               subject  \
0    Urgent: Critical impact on enterprise network ...   
1       Intermittent Cursor Freezing Issue on Dell XPS   
2      Exchange Request for Malfunctioning Dell XPS 13   
3         Request for Server Administration Assistance   
4     Immediate Attention Required: AWS Outage Concern   
..                                                 ...   
334                          AWS Authentication Issues   
335  Immediate Assistance Needed: Issues with Serve...   
336       High Priority: Recurring Network Disruptions   
337                                Request for Support   
338                     Immediate Assistance Requested   

                      product_tags  \
0      [Cisco Router ISR4331, WAN]   
1                               []   
2                           [Dell]   
3          [IT Consulting Service]   
4    [AWS Management Service, AWS]   
..                             ...   
3

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from collections import Counter
import spacy

# Load your dataset
file_path = '/content/helpdesk_customer_multi_lang_tickets.csv'
df = pd.read_csv(file_path)

# Filter for English tickets
df = df[df['language'] == 'en']

# Combine 'subject' and 'body' for clustering
df['combined_text'] = df['subject'].astype(str) + " " + df['body'].astype(str)

# Step 1: Cluster the data
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
kmeans = KMeans(n_clusters=5, random_state=42)
pipeline = make_pipeline(vectorizer, kmeans)
df['cluster_id'] = pipeline.fit_predict(df['combined_text'])

# Step 2: Generate Product Tags for Clusters
def get_top_tags(texts, top_n=10):
    all_words = " ".join(texts).split()
    most_common = Counter(all_words).most_common(top_n)
    return {word: count / len(texts) * 100 for word, count in most_common}

cluster_tags = {}
for cluster_id in sorted(df['cluster_id'].unique()):
    cluster_texts = df[df['cluster_id'] == cluster_id]['combined_text']
    cluster_tags[cluster_id] = get_top_tags(cluster_texts)

# Step 3: Extract Issues for Each Ticket
nlp = spacy.load('en_core_web_sm')

def extract_issues(text):
    doc = nlp(text)
    issues = [chunk.text for chunk in doc.noun_chunks if "issue" in chunk.text.lower()]
    return issues

df['issues'] = df['combined_text'].apply(extract_issues)

# Step 4: Display Results
print("Cluster Tags (with %):", cluster_tags)
print("Sample Issues Extracted:", df[['subject', 'issues']].head())


Cluster Tags (with %): {0: {'I': 465.57377049180326, 'the': 427.8688524590164, 'to': 411.47540983606564, 'and': 242.62295081967213, 'for': 237.7049180327869, 'this': 221.31147540983608, 'my': 206.55737704918033, 'you': 201.63934426229505, 'your': 201.63934426229505, 'a': 195.08196721311475}, 1: {'to': 475.8241758241758, 'the': 361.53846153846155, 'I': 287.9120879120879, 'and': 271.42857142857144, 'for': 225.27472527472528, 'this': 213.1868131868132, 'you': 203.2967032967033, 'your': 193.4065934065934, 'our': 179.12087912087912, 'is': 137.36263736263737}, 2: {'to': 386.15384615384613, 'the': 256.92307692307696, 'and': 207.6923076923077, 'our': 201.53846153846155, 'this': 193.84615384615384, 'I': 153.84615384615387, 'your': 146.15384615384613, 'you': 132.3076923076923, 'for': 127.69230769230768, 'with': 120.0}, 3: {'to': 467.3469387755102, 'and': 312.2448979591837, 'our': 267.34693877551024, 'your': 240.81632653061226, 'for': 210.20408163265304, 'the': 208.16326530612247, 'this': 165.306

# AFTER STOPWORD REMOVEL

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import defaultdict
import pandas as pd
import spacy

# Load the dataset
file_path = '/content/helpdesk_customer_multi_lang_tickets.csv'
df = pd.read_csv(file_path)

# Filter for English language
df = df[df['language'] == 'en']

# Combine 'subject' and 'body' for analysis
df['combined_text'] = df['subject'].astype(str) + " " + df['body'].astype(str)

# Step 1: Preprocess Text (Stopwords Removal & Lemmatization)
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text.lower())  # Convert to lowercase
    tokens = [
        token.lemma_
        for token in doc
        if not token.is_stop and not token.is_punct
    ]  # Remove stopwords and punctuation, apply lemmatization
    return " ".join(tokens)

df['processed_text'] = df['combined_text'].apply(preprocess_text)

# Step 2: Extract Product Names Using NER
def extract_product_names(text):
    doc = nlp(text)
    product_names = [ent.text for ent in doc.ents if ent.label_ in ['PRODUCT', 'ORG']]
    return product_names if product_names else ["Unknown Product"]

df['product_names'] = df['processed_text'].apply(extract_product_names)

# Step 3: Create Clusters Based on Processed Text
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['processed_text'])

kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster_id'] = kmeans.fit_predict(X)

# Step 4: Extract Issues for Each Ticket
def extract_issues(text):
    doc = nlp(text)
    issues = [chunk.text for chunk in doc.noun_chunks if 'issue' in chunk.text.lower()]
    return issues

df['issues'] = df['processed_text'].apply(extract_issues)

# Step 5: Aggregate Clusters with Product Tags and Issues
clusters = defaultdict(dict)
for cluster_id, group in df.groupby('cluster_id'):
    clusters[cluster_id]['tags'] = list(group['product_names'].explode().unique())
    clusters[cluster_id]['issues'] = list(group['issues'])

# Step 6: Display Results
print("Clusters with Product Tags and Issues:")
for cluster_id, details in clusters.items():
    print(f"Cluster {cluster_id}:")
    print(f"  Tags: {details['tags']}")
    print(f"  Issues: {details['issues'][:5]}")  # Display sample issues


Clusters with Product Tags and Issues:
Cluster 0:
  Tags: ['xps', '13 9310', 'Unknown Product', 'mg3620', 'request urgent assistance dell', 'xps 13', 'microsoft', 'et-4760', '9310', '5.11.0', 'java development task application', 'patch alleviate frequent disruption']
  Issues: [['issue dell', 'issue dell'], [], ['recent update frontend application issue pertain javascript', 'user experience \n\n specific issue component', 'issue', 'prior implement update \n\n provide guidance resolve issue early convenience problem'], ['issue step'], ['13 issue', 'issue', 'regain functionality issue']]
Cluster 1:
  Tags: ['Unknown Product', 'microsoft', 'urgent aws management service issue', 'urgent care need issue aw deployment customer face', 'urgent frequent postgresql', 'google', 'dns', 'samsung qle tv', 'urgent aws management service dear', '5.11.0', 'urgent consulting require server', 'log detail require', 'urgent issue aw deployment dear', 'urgent zoom application']
  Issues: [['request professi

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import defaultdict
import pandas as pd
import spacy

# Load the dataset
file_path = '/content/helpdesk_customer_multi_lang_tickets.csv'
df = pd.read_csv(file_path)

# Filter for English language
df = df[df['language'] == 'en']

# Combine 'subject' and 'body' for analysis
df['combined_text'] = df['subject'].astype(str) + " " + df['body'].astype(str)

# Step 1: Preprocess Text (Stopwords Removal & Lemmatization)
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text.lower())  # Convert to lowercase
    tokens = [
        token.lemma_
        for token in doc
        if not token.is_stop and not token.is_punct
    ]  # Remove stopwords and punctuation, apply lemmatization
    return " ".join(tokens)

df['processed_text'] = df['combined_text'].apply(preprocess_text)

# Step 2: Extract Product Names Using NER
def extract_product_names(text):
    doc = nlp(text)
    product_names = [ent.text for ent in doc.ents if ent.label_ in ['PRODUCT', 'ORG']]
    return product_names if product_names else ["Unknown Product"]

df['product_names'] = df['processed_text'].apply(extract_product_names)

# Step 3: Create Clusters Based on Processed Text
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['processed_text'])

kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster_id'] = kmeans.fit_predict(X)

# Step 4: Extract Issues for Each Ticket
def extract_issues(text):
    doc = nlp(text)
    issues = [chunk.text for chunk in doc.noun_chunks if 'issue' in chunk.text.lower()]
    return issues

df['issues'] = df['processed_text'].apply(extract_issues)

# Step 5: Aggregate Clusters with Product Tags and % Occurrence
clusters = defaultdict(dict)

for cluster_id, group in df.groupby('cluster_id'):
    # Count total tickets in the cluster
    total_tickets = len(group)

    # Flatten and count product name occurrences
    product_tags = group['product_names'].explode()
    tag_counts = product_tags.value_counts(normalize=True) * 100  # Normalize for percentages

    clusters[cluster_id]['tags'] = tag_counts.to_dict()
    clusters[cluster_id]['issues'] = list(group['issues'])

# Step 6: Display Results
print("Clusters with Product Tags and % Occurrence:")
for cluster_id, details in clusters.items():
    print(f"Cluster {cluster_id}:")
    print("  Tags with % Occurrence:")
    for tag, percentage in details['tags'].items():
        print(f"    {tag}: {percentage:.2f}%")
    print(f"  Sample Issues: {details['issues'][:5]}")  # Display sample issues


Clusters with Product Tags and % Occurrence:
Cluster 0:
  Tags with % Occurrence:
    xps: 23.48%
    13 9310: 21.74%
    microsoft: 20.00%
    Unknown Product: 11.30%
    mg3620: 10.43%
    et-4760: 6.96%
    xps 13: 1.74%
    request urgent assistance dell: 0.87%
    9310: 0.87%
    5.11.0: 0.87%
    java development task application: 0.87%
    patch alleviate frequent disruption: 0.87%
  Sample Issues: [['issue dell', 'issue dell'], [], ['recent update frontend application issue pertain javascript', 'user experience \n\n specific issue component', 'issue', 'prior implement update \n\n provide guidance resolve issue early convenience problem'], ['issue step'], ['13 issue', 'issue', 'regain functionality issue']]
Cluster 1:
  Tags with % Occurrence:
    Unknown Product: 86.86%
    microsoft: 4.00%
    dns: 1.71%
    google: 1.14%
    5.11.0: 1.14%
    urgent aws management service issue: 0.57%
    urgent care need issue aw deployment customer face: 0.57%
    urgent frequent postgresql

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import defaultdict
import pandas as pd
import spacy

# Load the dataset
file_path = '/content/helpdesk_customer_multi_lang_tickets.csv'
df = pd.read_csv(file_path)

# Filter for English language
df = df[df['language'] == 'en']

# Combine 'subject' and 'body' for analysis
df['combined_text'] = df['subject'].astype(str) + " " + df['body'].astype(str)

# Step 1: Preprocess Text (Stopwords Removal & Lemmatization)
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text.lower())  # Convert to lowercase
    tokens = [
        token.lemma_
        for token in doc
        if not token.is_stop and not token.is_punct
    ]  # Remove stopwords and punctuation, apply lemmatization
    return " ".join(tokens)

df['processed_text'] = df['combined_text'].apply(preprocess_text)

# Step 2: Extract Product Names Using Refined NER
def extract_product_names(text):
    doc = nlp(text)
    product_names = [
        ent.text
        for ent in doc.ents
        if ent.label_ in ['PRODUCT', 'ORG'] and len(ent.text.split()) <= 3
    ]  # Filter valid products/orgs
    return product_names if product_names else ["Unknown Product"]

df['product_names'] = df['processed_text'].apply(extract_product_names)

# Step 3: Create Clusters Based on Processed Text
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['processed_text'])

kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster_id'] = kmeans.fit_predict(X)

# Step 4: Extract Issues for Each Ticket
def extract_issues(text):
    doc = nlp(text)
    issues = [
        chunk.text
        for chunk in doc.noun_chunks
        if "issue" in chunk.text.lower() or "problem" in chunk.text.lower()
    ]
    return issues

df['issues'] = df['processed_text'].apply(extract_issues)

# Step 5: Aggregate Clusters with Product Tags and Issues
clusters = defaultdict(dict)
for cluster_id, group in df.groupby('cluster_id'):
    tag_counts = group['product_names'].explode().value_counts(normalize=True) * 100
    clusters[cluster_id]['tags'] = tag_counts.to_dict()
    clusters[cluster_id]['issues'] = group['issues'].explode().dropna().unique().tolist()

# Step 6: Display Results
print("Clusters with Product Tags and % Occurrence:")
for cluster_id, details in clusters.items():
    print(f"Cluster {cluster_id}:")
    print("  Tags with % Occurrence:")
    for tag, percentage in details['tags'].items():
        print(f"    {tag}: {percentage:.2f}%")
    print("  Sample Issues:")
    print(details['issues'][:5])  # Display sample issues


Clusters with Product Tags and % Occurrence:
Cluster 0:
  Tags with % Occurrence:
    xps: 23.68%
    13 9310: 21.93%
    microsoft: 20.18%
    Unknown Product: 13.16%
    mg3620: 10.53%
    et-4760: 7.02%
    xps 13: 1.75%
    9310: 0.88%
    5.11.0: 0.88%
  Sample Issues:
['issue dell', 'recent update frontend application issue pertain javascript', 'user experience \n\n specific issue component', 'issue', 'prior implement update \n\n provide guidance resolve issue early convenience problem']
Cluster 1:
  Tags with % Occurrence:
    Unknown Product: 89.71%
    microsoft: 4.00%
    dns: 1.71%
    google: 1.14%
    5.11.0: 1.14%
    urgent frequent postgresql: 0.57%
    samsung qle tv: 0.57%
    log detail require: 0.57%
    urgent zoom application: 0.57%
  Sample Issues:
['request professional assistance server administration software development environment client < > experience performance issue', 'bottleneck current configuration contribute performance issue', 'urgent problem', 'min